# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("./data/final_train_data_ver2.csv")
test_data = pd.read_csv("./data/final_test_data_ver2.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result_encoded',
    'WorkMode Collect Result',
    'Receip_n_suffix_3',
    'time_gap_All'
]

In [4]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 44
train_data_dam DataFrame의 칼럼 수: 25
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 16
train_data_fill2 DataFrame의 칼럼 수: 16
----test data-----
test_data DataFrame의 칼럼 수: 45
test_data_dam DataFrame의 칼럼 수: 26
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 17
test_data_fill2 DataFrame의 칼럼 수: 17


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [16]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [17]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    
    # 'Normal'과 'AbNormal'을 숫자로 변환
    y_tr = y_tr.map({'Normal': 0, 'AbNormal': 1})
    y_val = y_val.map({'Normal': 0, 'AbNormal': 1})
    
    param = {
        'iterations': trial.suggest_int('iterations', 800, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'depth': trial.suggest_int('depth', 4, 13),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 128, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        'random_seed': RANDOM_STATE,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }


    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill2.drop("target", axis=1), 
    train_data_fill2["target"],              
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=400)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-08-26 04:19:01,832] A new study created in memory with name: no-name-029e0791-6fea-4ad6-aaa9-5fb7b9760925
[I 2024-08-26 04:19:07,326] Trial 0 finished with value: 0.17073170731707316 and parameters: {'iterations': 848, 'learning_rate': 0.13339879944049993, 'depth': 7, 'l2_leaf_reg': 1.099256962998379, 'random_strength': 8.185937574992959, 'bagging_temperature': 6.85638497930364, 'border_count': 338, 'scale_pos_weight': 6.735006454525373, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.17073170731707316.
[I 2024-08-26 04:19:14,101] Trial 1 finished with value: 0.20552147239263804 and parameters: {'iterations': 1371, 'learning_rate': 0.031403208613589216, 'depth': 3, 'l2_leaf_reg': 0.5027927228085797, 'random_strength': 5.961831701274088, 'bagging_temperature': 0.31183600257184896, 'border_count': 227, 'scale_pos_weight': 1.698550183782988, 'grow_policy': 'Lossguide'}. Best is trial 1 with value: 0.20552147239263804.
[I 2024-08-26 04:19:25,387] Trial 2 finished wit

[I 2024-08-26 04:27:52,239] Trial 20 finished with value: 0.1871101871101871 and parameters: {'iterations': 2079, 'learning_rate': 0.07833476292419154, 'depth': 6, 'l2_leaf_reg': 1.6980906710840935, 'random_strength': 4.4814478390215955, 'bagging_temperature': 1.4265802934728589, 'border_count': 106, 'scale_pos_weight': 3.4054211411074755, 'grow_policy': 'Depthwise'}. Best is trial 13 with value: 0.21834061135371177.
[I 2024-08-26 04:28:00,075] Trial 21 finished with value: 0.20467836257309943 and parameters: {'iterations': 1589, 'learning_rate': 0.021368534455665816, 'depth': 3, 'l2_leaf_reg': 0.546617532933024, 'random_strength': 6.069496254444898, 'bagging_temperature': 0.7929899753984693, 'border_count': 213, 'scale_pos_weight': 1.8951360921994538, 'grow_policy': 'Lossguide'}. Best is trial 13 with value: 0.21834061135371177.
[I 2024-08-26 04:28:09,111] Trial 22 finished with value: 0.21629629629629632 and parameters: {'iterations': 1482, 'learning_rate': 0.025533039996572054, 'dep

[I 2024-08-26 04:32:34,419] Trial 40 finished with value: 0.20049504950495048 and parameters: {'iterations': 2395, 'learning_rate': 0.013724576502456363, 'depth': 8, 'l2_leaf_reg': 0.6111157966141966, 'random_strength': 1.608776753751552, 'bagging_temperature': 5.977290700746135, 'border_count': 199, 'scale_pos_weight': 2.4359122475658586, 'grow_policy': 'Lossguide'}. Best is trial 33 with value: 0.22539229671897287.
[I 2024-08-26 04:32:38,767] Trial 41 finished with value: 0.1606686788948224 and parameters: {'iterations': 1272, 'learning_rate': 0.031196780242549134, 'depth': 4, 'l2_leaf_reg': 2.0204240300695555, 'random_strength': 7.314359704186016, 'bagging_temperature': 2.788361441269437, 'border_count': 218, 'scale_pos_weight': 9.758876338884903, 'grow_policy': 'SymmetricTree'}. Best is trial 33 with value: 0.22539229671897287.
[I 2024-08-26 04:32:43,146] Trial 42 finished with value: 0.1991465149359886 and parameters: {'iterations': 1023, 'learning_rate': 0.03772938680013555, 'dep

[I 2024-08-26 04:36:12,088] Trial 60 finished with value: 0.19768934531450577 and parameters: {'iterations': 3177, 'learning_rate': 0.02234427456431744, 'depth': 11, 'l2_leaf_reg': 0.10006261875806695, 'random_strength': 11.7201737958028, 'bagging_temperature': 4.891080186126107, 'border_count': 264, 'scale_pos_weight': 2.274827915733778, 'grow_policy': 'Lossguide'}. Best is trial 33 with value: 0.22539229671897287.
[I 2024-08-26 04:36:32,882] Trial 61 finished with value: 0.18156028368794327 and parameters: {'iterations': 2906, 'learning_rate': 0.02775649424903602, 'depth': 10, 'l2_leaf_reg': 0.1864100164513773, 'random_strength': 0.8086245162581711, 'bagging_temperature': 4.860511823388056, 'border_count': 308, 'scale_pos_weight': 1.7651929124580004, 'grow_policy': 'Lossguide'}. Best is trial 33 with value: 0.22539229671897287.
[I 2024-08-26 04:36:48,890] Trial 62 finished with value: 0.19587628865979384 and parameters: {'iterations': 2241, 'learning_rate': 0.053112844819440445, 'dep

[I 2024-08-26 04:40:43,558] Trial 80 finished with value: 0.19565217391304346 and parameters: {'iterations': 2354, 'learning_rate': 0.010671076651608534, 'depth': 8, 'l2_leaf_reg': 0.10685866788835727, 'random_strength': 10.627749495516671, 'bagging_temperature': 0.7829432592613319, 'border_count': 206, 'scale_pos_weight': 3.5635156942048285, 'grow_policy': 'Depthwise'}. Best is trial 33 with value: 0.22539229671897287.
[I 2024-08-26 04:40:48,803] Trial 81 finished with value: 0.21 and parameters: {'iterations': 1233, 'learning_rate': 0.03126819952637053, 'depth': 6, 'l2_leaf_reg': 0.2054564801725654, 'random_strength': 7.198744088576087, 'bagging_temperature': 0.11456814524896475, 'border_count': 185, 'scale_pos_weight': 2.3365973594949354, 'grow_policy': 'SymmetricTree'}. Best is trial 33 with value: 0.22539229671897287.
[I 2024-08-26 04:40:54,519] Trial 82 finished with value: 0.21660649819494587 and parameters: {'iterations': 1356, 'learning_rate': 0.019366219070503326, 'depth': 6,

...

[I 2024-08-26 04:57:57,067] Trial 320 finished with value: 0.2219873150105708 and parameters: {'iterations': 1092, 'learning_rate': 0.011398965665058916, 'depth': 3, 'l2_leaf_reg': 0.16075880996571298, 'random_strength': 10.947646107851725, 'bagging_temperature': 8.137512836938502, 'border_count': 85, 'scale_pos_weight': 4.098110319543926, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 04:58:01,339] Trial 321 finished with value: 0.23118696664080682 and parameters: {'iterations': 1159, 'learning_rate': 0.005136586249816276, 'depth': 5, 'l2_leaf_reg': 0.17752177650872128, 'random_strength': 10.752183482058, 'bagging_temperature': 8.275203469825291, 'border_count': 98, 'scale_pos_weight': 3.239289138263469, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 04:58:04,551] Trial 322 finished with value: 0.23177842565597667 and parameters: {'iterations': 931, 'learning_rate': 0.0085080457010623

[I 2024-08-26 04:58:53,683] Trial 340 finished with value: 0.21387283236994223 and parameters: {'iterations': 648, 'learning_rate': 0.15227576181734648, 'depth': 3, 'l2_leaf_reg': 0.17501760146558534, 'random_strength': 11.530562673651284, 'bagging_temperature': 5.470328886853444, 'border_count': 89, 'scale_pos_weight': 3.4043570436464274, 'grow_policy': 'Depthwise'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 04:58:56,183] Trial 341 finished with value: 0.22816679779701024 and parameters: {'iterations': 794, 'learning_rate': 0.014593720390670977, 'depth': 3, 'l2_leaf_reg': 0.13483369081737961, 'random_strength': 11.750669262234352, 'bagging_temperature': 4.975296817409518, 'border_count': 100, 'scale_pos_weight': 3.161681123072831, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 04:58:58,951] Trial 342 finished with value: 0.22954977805960683 and parameters: {'iterations': 883, 'learning_rate': 0.01119890358657367

[I 2024-08-26 05:00:04,935] Trial 360 finished with value: 0.23313782991202345 and parameters: {'iterations': 913, 'learning_rate': 0.008691491478061978, 'depth': 3, 'l2_leaf_reg': 0.10530966835244288, 'random_strength': 11.161459055170061, 'bagging_temperature': 7.058331248811092, 'border_count': 88, 'scale_pos_weight': 3.3695336938960656, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 05:00:07,999] Trial 361 finished with value: 0.22847682119205298 and parameters: {'iterations': 986, 'learning_rate': 0.022347781777576937, 'depth': 3, 'l2_leaf_reg': 0.40359093653955946, 'random_strength': 11.526712031872206, 'bagging_temperature': 5.051493700662351, 'border_count': 96, 'scale_pos_weight': 3.0723838710195404, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 05:00:10,641] Trial 362 finished with value: 0.22366097704532076 and parameters: {'iterations': 844, 'learning_rate': 0.011837968379

[I 2024-08-26 05:01:57,067] Trial 380 finished with value: 0.23333333333333334 and parameters: {'iterations': 1115, 'learning_rate': 0.007681524213697887, 'depth': 4, 'l2_leaf_reg': 0.10021122117490561, 'random_strength': 10.700350519325061, 'bagging_temperature': 4.491838174954221, 'border_count': 100, 'scale_pos_weight': 3.2891666989048627, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 05:01:59,763] Trial 381 finished with value: 0.22885572139303478 and parameters: {'iterations': 857, 'learning_rate': 0.0051808694394642665, 'depth': 3, 'l2_leaf_reg': 0.17001545338574883, 'random_strength': 11.231493213401976, 'bagging_temperature': 8.052545559186916, 'border_count': 80, 'scale_pos_weight': 3.5243394863555753, 'grow_policy': 'SymmetricTree'}. Best is trial 117 with value: 0.24251069900142652.
[I 2024-08-26 05:02:04,807] Trial 382 finished with value: 0.2198053806525472 and parameters: {'iterations': 991, 'learning_rate': 0.0118419112

Best trial: score 0.24251069900142652, 
params {'iterations': 987, 'learning_rate': 0.011942535987477916, 'depth': 4, 'l2_leaf_reg': 0.17913310604185315, 'random_strength': 11.61512820207189, 'bagging_temperature': 7.9524408302966885, 'border_count': 143, 'scale_pos_weight': 3.361391569468003, 'grow_policy': 'SymmetricTree'}


1. 0.24251069900142652

params {'iterations': 987, 'learning_rate': 0.011942535987477916, 'depth': 4, 'l2_leaf_reg': 0.17913310604185315, 'random_strength': 11.61512820207189, 'bagging_temperature': 7.9524408302966885, 'border_count': 143, 'scale_pos_weight': 3.361391569468003, 'grow_policy': 'SymmetricTree'}

In [13]:
# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

In [14]:
train_and_evaluate_model(
    'cat', train_data_fill2,
    iterations = 481, 
    learning_rate = 0.018742270357007457, 
    depth = 5, 
    min_data_in_leaf = ,
    l2_leaf_reg = 1.0871571324663387, 
    random_strength = 3.49632241801363, 
    bagging_temperature = 5.717049796462913, 
    border_count = 183, 
    scale_pos_weight = 3.4406776189795383,
    grow_policy = 'SymmetricTree',


    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

cat 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.24322446143154972
---
Confusion Matrix:
[[6838  824]
 [ 265  175]]
---
Accuracy: 0.8655887435201185
Precision: 0.17517517517517517
Recall: 0.3977272727272727


